In [1]:
import sys,os
os.chdir('../../')
sys.path.append('./src/data')
sys.path.append('./src/events')
sys.path.append('./src/probability')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from importer_gen2 import *
from processer_gen2 import *
from gen2.main import get_all_events,get_events
from functions import perform_chisq, dc_params_nsi, dc_params
from gen2.event_processing import list_of_params_nsi, return_precomputed_nsi, get_deltachi
from scipy.stats import chi2
import pickle
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams['figure.dpi'] = 100
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')
np.set_printoptions(linewidth=200)

1.4737010682019581e-05


In [37]:
ndim = 3
pid = 1
precomputed_events = True
dc_params_nsi['dm_41'] = 0.93
dc_params_nsi['theta_12'] = 0.5839958715755919
dc_params['dm_41'] = 0.93
emm_range = np.linspace(0,0,1)
s24_range = np.linspace(0,0,1)
emt_range = np.linspace(-1e-2,1e-2,100)
param_list = list_of_params_nsi(dc_params_nsi,s24_range, emm_range,emt_range)

#for p in param_list: # Assert all dicts returned from param_list have precomputed probs.
#    assert is_precomputed_nsi(N=N,ndim=ndim, dict=p,check=False)



param_list = return_precomputed_nsi(pid,ndim,params=param_list)
emm_range = np.sort(np.unique(np.array([p['e_mm'] for p in param_list])))
s24_range = np.sin(2*np.sort(np.unique(np.array([p['theta_24'] for p in param_list]))))**2
emt_range = np.sort(np.unique(np.array([p['e_mt'] for p in param_list])))

print(emm_range)
print(s24_range)
print(emt_range)

#print(param_list[1])

[0.]
[0.]
[-0.01       -0.00979798 -0.00959596 -0.00939394 -0.00919192 -0.0089899  -0.00878788 -0.00858586 -0.00838384 -0.00818182 -0.0079798  -0.00777778 -0.00757576 -0.00737374 -0.00717172 -0.0069697
 -0.00676768 -0.00656566 -0.00636364 -0.00616162 -0.0059596  -0.00575758 -0.00555556 -0.00535354 -0.00515152 -0.00494949 -0.00474747 -0.00454545 -0.00434343 -0.00414141 -0.00393939 -0.00373737
 -0.00353535 -0.00333333 -0.00313131 -0.00292929 -0.00272727 -0.00252525 -0.00232323 -0.00212121 -0.00191919 -0.00171717 -0.00151515 -0.00131313 -0.00111111 -0.00090909 -0.00070707 -0.00050505
 -0.00030303 -0.00010101  0.00010101  0.00030303  0.00050505  0.00070707  0.00090909  0.00111111  0.00131313  0.00151515  0.00171717  0.00191919  0.00212121  0.00232323  0.00252525  0.00272727
  0.00292929  0.00313131  0.00333333  0.00353535  0.00373737  0.00393939  0.00414141  0.00434343  0.00454545  0.00474747  0.00494949  0.00515152  0.00535354  0.00555556  0.00575758  0.0059596
  0.00616162  0.00636364  0

In [3]:
if not precomputed_events:
    print('Computing events')
    from multiprocessing import Pool
    p = Pool()
    data = [(p, pid,False) for p in param_list]
    H1_events_list = p.starmap(get_all_events, data)
    p.close()
    pickle.dump(H1_events_list,open(f'./pre_computed/H1_gen2_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','wb'))

    H0_events = get_all_events(params=param_list[0], pid=pid, null=True)
    pickle.dump(H0_events,open(f'./pre_computed/H0_gen2_{pid}.p','wb'))

H1_events_list = pickle.load(open(f'./pre_computed/H1_gen2_{pid}_{len(emm_range)}x{len(emt_range)}_tau_nsi.p','rb'))
H0_events = pickle.load(open(f'./pre_computed/H0_gen2_{pid}.p','rb'))


#H0_normalized, H1_list_normalized = normalize_events(H0_events,H1_events_list,z_bins)

In [13]:
sigma_a = 0.25
sigma_b = 0.04
f = 0
deltachi, best_index, H1_chisq = get_deltachi(np.array(H1_events_list)*365*24*36, H0_events*365*24*36,emt_range,emm_range,0,[sigma_a,sigma_b],f,x0=[1])
#s24_cl90, s24_cl99, emm_cl90, emm_cl99 = get_contour(deltachi.T, emt_range,emm_range,2)
#best_fit_coords = (param_list[best_index]['e_mt'], param_list[best_index]['e_mm'])

In [11]:
H1_events_list[0]*365*24*36

array([[ 472.72296681,  501.74185901,  581.19196066,  594.5718119 ],
       [ 683.68711958,  713.52312682,  772.40801843, 1127.76474729],
       [ 804.0815826 ,  750.40983627,  916.71137103,  970.56981414],
       [ 847.49747417,  832.9879473 , 1141.34029623, 1062.56198819],
       [ 672.2914137 ,  801.02471031,  875.15237165, 1027.81510117],
       [ 519.35066634,  538.62858339,  551.90021868,  668.5348658 ],
       [ 304.02759975,  276.5840404 ,  333.29177351,  448.42034768],
       [ 178.34275444,  167.9513934 ,  218.77573107,  254.17441502]])

In [8]:
H0_events

array([[1.54953779e-05, 1.67683068e-05, 1.96054121e-05, 2.04921239e-05],
       [1.69160798e-05, 1.98138840e-05, 2.11874257e-05, 2.71371796e-05],
       [1.72529627e-05, 1.81911554e-05, 2.16411563e-05, 2.30337670e-05],
       [1.63814954e-05, 1.72979581e-05, 2.24107279e-05, 2.41506551e-05],
       [1.40820695e-05, 1.65761610e-05, 1.90702847e-05, 2.23231541e-05],
       [1.13837635e-05, 1.27295146e-05, 1.43612795e-05, 1.73486785e-05],
       [8.33621718e-06, 8.82926501e-06, 1.09535875e-05, 1.44457003e-05],
       [6.24343666e-06, 6.47724822e-06, 8.55290024e-06, 1.06007423e-05]])

In [20]:
from gen2.event_processing import perform_chisq
np.array([perform_chisq(H1_norm, H0_events,sigma_syst=0,z=0, sigma_a=0.25,sigma_b=0.04,sigma_g=0, x0=[1])[0] for H1_norm in H1_events_list[0:3]])

array([0.98533481, 0.98533481, 0.98533481])

In [19]:
H1_events_list[10]

array([[0.00149899, 0.00159101, 0.00184295, 0.00188537],
       [0.00216796, 0.00226257, 0.00244929, 0.00357612],
       [0.00254973, 0.00237953, 0.00290687, 0.00307766],
       [0.0026874 , 0.00264139, 0.00361917, 0.00336936],
       [0.00213182, 0.00254003, 0.00277509, 0.00325918],
       [0.00164685, 0.00170798, 0.00175006, 0.00211991],
       [0.00096407, 0.00087704, 0.00105686, 0.00142193],
       [0.00056552, 0.00053257, 0.00069373, 0.00080598]])

In [42]:
from numerical import wrapper
E_range=[1e2,1e3]
z_range=[-0.9,-0.5]
np.array([wrapper(['m', [E_range[i]],z, False, param_list[10], 3, True])[1] for i,z in enumerate(z_range)])

array([[0.95570011],
       [0.99955988]])

In [44]:
from processer_gen2 import generate_probabilities_gen2
generate_probabilities_gen2('m', 'm', E_range,z_range,0,0,param_list[10],False,1, ndim=3, nsi=True)

3gen/Pmm/1/e1ad9cebff03e722e6200d1f5bc06619e684536b57eb501210ab6a95f2f86f6e already exists, skipping


array([[0.95570011],
       [0.99955988]])